# Lab 10 > Part A

> Name: Debatreya Das <br>
> Roll No. 12212070 <br>
> CS A4 <br>
> Data Mining Lab 10

### Create A data warehouse from different .csv files using PostgreSQL tool

## Dataset

I have gathered 3 CSV files.
- [products.csv](./Data/products.csv)
- [sales.csv](./Data/sales.csv)
- [customers.csv](./Data/customer.csv)

Now I will use Python to Put these Tables in Postgres and then Show Applying a Query

## Import Libraries and Connect to PostgreSQL

In [2]:
import psycopg2
import pandas as pd

In [21]:
# Connect with local Postgres
conn = psycopg2.connect(
    dbname="warehouse", 
    user="postgres",
    password="postgres", 
    host="localhost"
)
cursor = conn.cursor()

### Create Tables

In [22]:
# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS sales;")
cursor.execute("DROP TABLE IF EXISTS customers;")
cursor.execute("DROP TABLE IF EXISTS products;")

# Create 'products' table
cursor.execute("""
CREATE TABLE products (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(255),
    category VARCHAR(255),
    price NUMERIC
);
""")

# Create 'customers' table
cursor.execute("""
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    city VARCHAR(255)
);
""")

# Create 'sales' table
cursor.execute("""
CREATE TABLE sales (
    sale_id SERIAL PRIMARY KEY,
    product_id INT REFERENCES products(product_id),
    customer_id INT REFERENCES customers(customer_id),
    sale_date DATE,
    quantity INT
);
""")

conn.commit()

### Load Data from CSV into Respective Files

In [23]:
# Load each .csv file into the respective table
with open('./Data/products.csv', 'r') as f:
    next(f)  # Skip the header row
    cursor.copy_from(f, 'products', sep=',')

with open('./Data/customer.csv', 'r') as f:
    next(f)
    cursor.copy_from(f, 'customers', sep=',')

with open('./Data/sales.csv', 'r') as f:
    next(f)
    cursor.copy_from(f, 'sales', sep=',')


conn.commit()

Now the Tables are ready.

## Quering the Data Warehouse

In [24]:
# Example query to retrieve sales with customer details
query = """
SELECT s.sale_id, s.sale_date, p.product_name, c.first_name, c.last_name, s.quantity, p.price
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN customers c ON s.customer_id = c.customer_id;
"""

# Run the query and display the results
df = pd.read_sql(query, conn)
df.head()

C:\Users\debat\AppData\Local\Temp\ipykernel_24728\2773805579.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,sale_id,sale_date,product_name,first_name,last_name,quantity,price
0,1,2024-01-01,Smartphone,John,Doe,2,500.0
1,2,2024-01-03,Blender,Jane,Smith,1,80.0
2,3,2024-01-05,Laptop,Mike,Brown,1,1200.0
3,4,2024-01-08,Watch,Sara,Johnson,3,150.0
4,5,2024-01-09,Shoes,John,Doe,1,60.0


### Close connection

In [25]:
# Close cursor and connection
cursor.close()
conn.close()